In [64]:
from api_tools import GPTPrompt, run_experiment, run_gpt_prompt, StringMetric

## Background
Children up to the age of 8 incorrectly coindex PRO with the subject in cases of object control, such as "x is easy to see".

## Does GPT-3 do this as well? 

In [65]:

replicants = 20
class_lookups = {"easy": ["easy"], "hard": ["hard", "difficult"]}
metric = StringMetric(class_lookups)
text1a =  """Answer the following question with either "Easy to see" or "Hard to see".\nQuestion: Is a person with a blindfold on easy to see, or hard to see?\nAnswer: """
kwargs = {"max_tokens": 8, "temperature": 0.1}
metric, responses = run_experiment(run_gpt_prompt, text1a, replicants, metric, kwargs)

print(f"PROMPT:\n{text1a}")
# print(f"RESPONSE:\n-----------{resp}")
counts, classes = metric.get_metric()


 45%|████▌     | 9/20 [00:06<00:08,  1.33it/s]


KeyboardInterrupt: 

In [50]:
print(counts)

# for c, resps in classes.items():
#    print(c)
#    for r in resps:
#        r = r.split("\n")[0]
#        print(f"\t{r}")

{'easy': 20, 'hard': 0, 'other': 0}


## Is it just picking the second option? 

In [48]:

replicants = 20
class_lookups = {"easy": ["easy"], "hard": ["hard", "difficult"]}
metric = StringMetric(class_lookups)
text1a =  """Answer the following question with either "Easy to see" or "Hard to see".\nQuestion: Is a person with a blindfold on hard to see, or easy to see?\nAnswer: """
kwargs = {"max_tokens": 8, "temperature": 0.1}
metric, responses = run_experiment(run_gpt_prompt, text1a, replicants, metric, kwargs)

print(f"PROMPT:\n{text1a}")
counts, classes = metric.get_metric()

100%|██████████| 20/20 [00:13<00:00,  1.47it/s]

PROMPT:
Answer the following question with either "Easy to see" or "Hard to see".
Question: Is a person with a blindfold on hard to see, or easy to see?
Answer: 


In [51]:
print(counts)

# for c, resps in classes.items():
#    print(c)
#    for r in resps:
#        r = r.split("\n")[0]
#        print(f"\t{r}")

{'easy': 20, 'hard': 0, 'other': 0}


## What if you prompt with other correct examples 

In [47]:

context = ["""Answer the following question with either "Easy to see" or "Hard to see".\n""",
           "Question: Is a child with a blindfold on easy to see, or hard to see?", "A: \" Easy to see . \"",
           "Question: Is a doll with a blindfold on easy to see, or hard to see?", "A: \" Easy to see . \"",
           "Question: Is a person with a blindfold on easy to see, or hard to see?"]
prompt_text = "Answer: "

prompt = GPTPrompt(context, prompt_text)

replicants = 20
class_lookups = {"easy": ["easy"], "hard": ["hard", "difficult"]}
metric = StringMetric(class_lookups)
text = str(prompt)
kwargs = {"max_tokens": 8, "temperature": 0.1}
metric, responses = run_experiment(run_gpt_prompt, text, replicants, metric, kwargs)
counts, classes = metric.get_metric()

print(counts)

100%|██████████| 20/20 [00:11<00:00,  1.71it/s]

{'easy': 0, 'hard': 20, 'other': 0}


## What if you prompt with incorrect examples

In [46]:

context = ["""Answer the following question with either "Easy to see" or "Hard to see".\n""",
           "Question: Is a child with a blindfold on easy to see, or hard to see?", "A: \" Hard to see . \"",
           "Question: Is a doll with a blindfold on easy to see, or hard to see?", "A: \" Hard to see . \"",
           "Question: Is a person with a blindfold on easy to see, or hard to see?"]
prompt_text = "Answer: "

prompt = GPTPrompt(context, prompt_text)

replicants = 20
class_lookups = {"easy": ["easy"], "hard": ["hard", "difficult"]}
metric = StringMetric(class_lookups)
text = str(prompt)
kwargs = {"max_tokens": 8, "temperature": 0.1}
metric, responses = run_experiment(run_gpt_prompt, text, replicants, metric, kwargs)
counts, classes = metric.get_metric()

print(counts)

100%|██████████| 20/20 [00:13<00:00,  1.53it/s]

{'easy': 20, 'hard': 0, 'other': 0}


## Conclusions so far 
1. Chooses "easy to see" always when unprompted
2. If prompted, it just chooses the opposite of what it's prompted with
    - if it's prompted with "easy to see" it always chooses "hard to see" and vice versa, so prompting isn't as interesting 
    - probably picking up on some heuristic from tests that the easy answer isn't the right one

## Does wording matter? 

In [45]:
context = ["""Answer the following question with either "Easy to see" or "Hard to see".\n""",
           "Question: Is a person wearing a blindfold easy to see, or hard to see?"]
prompt_text = "Answer: "

prompt = GPTPrompt(context, prompt_text)
replicants = 20
class_lookups = {"easy": ["easy"], "hard": ["hard", "difficult"]}
metric = StringMetric(class_lookups)
text = str(prompt)
kwargs = {"max_tokens": 8, "temperature": 0.1}
metric, responses = run_experiment(run_gpt_prompt, text, replicants, metric, kwargs)
counts, classes = metric.get_metric()

print(f"No priming: {counts}")

context = ["""Answer the following question with either "Easy to see" or "Hard to see".\n""",
           "Question: Is a child wearing blindfold easy to see, or hard to see?", "A: \" Easy to see . \"",
           "Question: Is a doll wearing blindfold easy to see, or hard to see?", "A: \" Easy to see . \"",
           "Question: Is a person wearing blindfold easy to see, or hard to see?"]
prompt_text = "Answer: "

prompt = GPTPrompt(context, prompt_text)
text = str(prompt)

replicants = 20
class_lookups = {"easy": ["easy"], "hard": ["hard", "difficult"]}
metric = StringMetric(class_lookups)
kwargs = {"max_tokens": 8, "temperature": 0.1}

metric, responses = run_experiment(run_gpt_prompt, text, replicants, metric, kwargs)
counts, classes = metric.get_metric()
resp = run_gpt_prompt(str(prompt), None)
counts, classes = metric.get_metric()

print(f"With priming: {counts}")


100%|██████████| 20/20 [00:13<00:00,  1.50it/s]


No priming: {'easy': 0, 'hard': 0, 'other': 20}


100%|██████████| 20/20 [00:12<00:00,  1.66it/s]


With priming: {'easy': 0, 'hard': 20, 'other': 0}


In [ ]:
print(classes)

## Does it hate Blindfolds

In [43]:
context = ["""Answer the following question with either "Easy to see" or "Hard to see".\n""",
           "Question: Are blind people easy to see, or hard to see?"]
prompt_text = "Answer: "

prompt = GPTPrompt(context, prompt_text)
replicants = 20
class_lookups = {"easy": ["easy"], "hard": ["hard", "difficult"]}
metric = StringMetric(class_lookups)
text = str(prompt)
kwargs = {"max_tokens": 8, "temperature": 0.1}
metric, responses = run_experiment(run_gpt_prompt, text, replicants, metric, kwargs)
counts, classes = metric.get_metric()

100%|██████████| 20/20 [00:13<00:00,  1.53it/s]


In [44]:
print(counts)
print(classes)

{'easy': 0, 'hard': 0, 'other': 20}
{'easy': [], 'hard': [], 'other': ['\n Answer :   \n \n Answer', '\n Answer  the  following  question  with  either', '__ \n \n Answer :   ________', '\n Answer :   \n \n Answer', '\n Answer  the  following  question  with  either', '\n Answer  the  following  question  with  either', '_______ \n \n Answer  the  following  question', '\n Answer  the  following  question  with  either', '\n Answer :   \n \n Answer', '\n Answer :   \n \n Answer', '\n Answer  the  following  question  with  either', '__ \n \n Answer :   ________', '\n Answer  the  following  question  with  either', '\n Answer :   \n \n Answer', '\n Answer :   \n \n Answer', '\n Answer  the  following  question  with  either', '\n Answer  the  following  question  with  either', '__ \n \n Answer :   ________', '\n Answer  the  following  question  with  either', '\n Answer  the  following  question  with  either']}


In [ ]:
print(prompt)

# What about "promising" vs "asking"

Now it's time to look at other stimuli, like "x promises y to do z"

In [41]:

context = ["""You will be given a context and a question. Answer the question with either "Tom" or "Mary".\nContext: Tom promised Mary to call home.\n""",
           "Question:  Who called home, Tom or Mary?"]
prompt_text = "Answer: "

prompt = GPTPrompt(context, prompt_text)

replicants = 20
class_lookups = {"Tom": ["tom"], "Mary": ["mary"]}
metric = StringMetric(class_lookups)
text = str(prompt)
kwargs = {"max_tokens": 2, "temperature": 0.0}
metric, responses = run_experiment(run_gpt_prompt, text, replicants, metric, kwargs)
counts, classes = metric.get_metric()

print(counts)

100%|██████████| 20/20 [00:06<00:00,  2.91it/s]

{'Tom': 20, 'Mary': 0, 'other': 0}


In [42]:

context = ["""You will be given a context and a question. Answer the question with either "Tom" or "Mary".\nContext: Tom asked Mary to call home.\n""",
           "Question:  Who called home, Tom or Mary?"]
prompt_text = "Answer: "

prompt = GPTPrompt(context, prompt_text)

replicants = 20
class_lookups = {"Tom": ["tom"], "Mary": ["mary"]}
metric = StringMetric(class_lookups)
text = str(prompt)
kwargs = {"max_tokens": 2, "temperature": 0.0}
metric, responses = run_experiment(run_gpt_prompt, text, replicants, metric, kwargs)
counts, classes = metric.get_metric()

print(counts)

100%|██████████| 20/20 [00:07<00:00,  2.72it/s]

{'Tom': 20, 'Mary': 0, 'other': 0}


## Switch the order of Tom and Mary

In [40]:

context = ["""You will be given a context and a question. Answer the question with either "Mary" or "Tom".\nContext: Mary promised Tom to call home.\n""",
           "Question:  Who called home, Mary or Tom?"]
prompt_text = "Answer: "

prompt = GPTPrompt(context, prompt_text)

replicants = 20
class_lookups = {"Tom": ["tom"], "Mary": ["mary"]}
metric = StringMetric(class_lookups)
text = str(prompt)
kwargs = {"max_tokens": 2, "temperature": 0.0}
metric, responses = run_experiment(run_gpt_prompt, text, replicants, metric, kwargs)
counts, classes = metric.get_metric()

print(counts)

100%|██████████| 20/20 [00:07<00:00,  2.71it/s]

{'Tom': 0, 'Mary': 14, 'other': 6}


In [39]:

context = ["""You will be given a context and a question. Answer the question with either "Mary" or "Tom".\nContext: Mary asked Tom to call home.\n""",
           "Question:  Who called home, Mary or Tom?"]
prompt_text = "Answer: "

prompt = GPTPrompt(context, prompt_text)

replicants = 20
class_lookups = {"Tom": ["tom"], "Mary": ["mary"]}
metric = StringMetric(class_lookups)
text = str(prompt)
kwargs = {"max_tokens": 2, "temperature": 0.0}
metric, responses = run_experiment(run_gpt_prompt, text, replicants, metric, kwargs)
counts, classes = metric.get_metric()

print(counts)

100%|██████████| 20/20 [00:07<00:00,  2.68it/s]

{'Tom': 0, 'Mary': 20, 'other': 0}


## Results
For ask versus promise, the model gets "promise" right, but "ask" wrong. Maybe it's getting confused by asked, as in "asked permission" rather than "asked for person 2 to do the thing" 

## Switch ask to force
force is higher factuality than ask

In [52]:

context = ["""You will be given a context and a question. Answer the question with either "Mary" or "Tom".\nContext: Mary forced Tom to call home.\n""",
           "Question:  Who called home, Mary or Tom?"]
prompt_text = "Answer: "

prompt = GPTPrompt(context, prompt_text)

replicants = 20
class_lookups = {"Tom": ["tom"], "Mary": ["mary"]}
metric = StringMetric(class_lookups)
text = str(prompt)
kwargs = {"max_tokens": 2, "temperature": 0.0}
metric, responses = run_experiment(run_gpt_prompt, text, replicants, metric, kwargs)
counts, classes = metric.get_metric()

print(counts)



context = ["""You will be given a context and a question. Answer the question with either "Tom" or "Mary".\nContext: Tom forced Mary to call home.\n""",
           "Question:  Who called home, Tom or Mary?"]
prompt_text = "Answer: "

prompt = GPTPrompt(context, prompt_text)

replicants = 20
class_lookups = {"Tom": ["tom"], "Mary": ["mary"]}
metric = StringMetric(class_lookups)
text = str(prompt)
kwargs = {"max_tokens": 2, "temperature": 0.0}
metric, responses = run_experiment(run_gpt_prompt, text, replicants, metric, kwargs)
counts, classes = metric.get_metric()

print(counts)

100%|██████████| 20/20 [00:07<00:00,  2.83it/s]


{'Tom': 0, 'Mary': 20, 'other': 0}


100%|██████████| 20/20 [00:06<00:00,  2.98it/s]

{'Tom': 18, 'Mary': 2, 'other': 0}


## Even with "force" it is conindexing with the subject rather than the object for object control 

## What about passives

In [53]:

context = ["""You will be given a context and a question. Answer the question with either "Mary" or "Tom".\nContext: Mary was forced by Tom to call home.\n""",
           "Question:  Who called home, Mary or Tom?"]
prompt_text = "Answer: "

prompt = GPTPrompt(context, prompt_text)

replicants = 20
class_lookups = {"Tom": ["tom"], "Mary": ["mary"]}
metric = StringMetric(class_lookups)
text = str(prompt)
kwargs = {"max_tokens": 2, "temperature": 0.0}
metric, responses = run_experiment(run_gpt_prompt, text, replicants, metric, kwargs)
counts, classes = metric.get_metric()

print(counts)



context = ["""You will be given a context and a question. Answer the question with either "Tom" or "Mary".\nContext: Tom was forced by Mary to call home.\n""",
           "Question:  Who called home, Tom or Mary?"]
prompt_text = "Answer: "

prompt = GPTPrompt(context, prompt_text)

replicants = 20
class_lookups = {"Tom": ["tom"], "Mary": ["mary"]}
metric = StringMetric(class_lookups)
text = str(prompt)
kwargs = {"max_tokens": 2, "temperature": 0.0}
metric, responses = run_experiment(run_gpt_prompt, text, replicants, metric, kwargs)
counts, classes = metric.get_metric()

print(counts)

100%|██████████| 20/20 [00:06<00:00,  2.91it/s]


{'Tom': 0, 'Mary': 20, 'other': 0}


100%|██████████| 20/20 [00:07<00:00,  2.83it/s]

{'Tom': 20, 'Mary': 0, 'other': 0}


## Conclusions

1. Doesn't have trouble with subject control in the case of "easy to see" and "hard to see" 
2. Unlike children, it has trouble with object control but not subject control 
    - MDP predicts that PRO will be co-indexed with the closest noun, e.g. that for "Tom promised/asked Mary to call home" it will be coindexed with Mary
    - The model actually does the opposite of what children do -- coindexes with the subject (maximally far away) instead of the object. So it gets subject control (promise) correct but object control (ask) wrong. 
3. In following with Maratsos, model gets passives correct
    - suggests that model is definitely not following the MDP 

## Follow-up question
Maratsos (1974) hypothesizes that the semantic roles in a sentence determine control. 

1. Is the evidence so far consistent with that hypothesis
    - easy vs hard stimuli: somewhat, in that it isn't getting them right all the time 
    - subject vs object control: could be consistent with a bias towards "agent-subject of the matrix clause is the agent-subject of the embedded clause" 
    - passives: **not** consistent with "agent of the matrix clause is agent of the embedded clause. Here, matrix patient-subject is the agent in second clause. 
2. How can we manipulate semantic roles here to get evidence on whether the model is using semantic role information 
    - right now we have 


    | matrix subj. | embedded subj. | experiment  | 
    | ------------ | ---------- | ----------- | 
    | agent | agent      | promise     | 
    | agent | patient | ask | 
    | patient | patient | passive told/asked | 

    - can we get patient agent? Something like "Tom was promised by Mary to call home" but grammatical 

## What about propose
propose is ambiguous between subject and object control
e.g. 
- "Tom proposed to Mary to leave" --> they both leave
- "Tom proposed to Bill to be his editor" --> either Bill or Tom is the editor
- "Tom proposed to Mary to be her editor" --> Tom is the editor (coerced by "her")
- "Tom proposed to Mary to be his editor" --> Mary is the editor (coerced by "his")


In [36]:

context = ["""You will be given a context and a question. Answer the question with "Tom", "Mary", or "both".\nContext: Tom proposed to Mary to leave.\n""",
           "Question: Who will leave, Tom, Mary, or both?"]
prompt_text = "Answer: "

prompt = Prompt(context, prompt_text)

replicants = 20
class_lookups = {"Tom": ["tom"], "Mary": ["mary"], "both": ["both"]}
metric = StringMetric(class_lookups)
text = str(prompt)
kwargs = {"max_tokens": 2, "temperature": 0.0}
metric, responses = run_experiment(text, replicants, metric, kwargs)
counts, classes = metric.get_metric()

print(counts)



100%|██████████| 20/20 [00:08<00:00,  2.47it/s]

{'Tom': 0, 'Mary': 0, 'both': 0, 'other': 20}


In [35]:

context = ["""You will be given a context and a question. Answer the following yes-no question.\nContext: Tom suggested Bill to be his editor.\n""",
           "Question: Will Tom be the editor?"]

prompt_text = "Answer: "

prompt = Prompt(context, prompt_text)

replicants = 20
class_lookups = {"Tom": ["tom", "yes"], "Bill": ["bill", "no"]}
metric = StringMetric(class_lookups)
text = str(prompt)
kwargs = {"max_tokens": 2, "temperature": 0.0}
metric, responses = run_experiment(text, replicants, metric, kwargs)
counts, classes = metric.get_metric()

print(counts)



100%|██████████| 20/20 [00:07<00:00,  2.59it/s]

{'Tom': 0, 'Bill': 0, 'other': 20}


In [ ]:
print(responses)

In [34]:
context = ["""You will be given a context and a question. Answer the question with either "Tom" or "Mary".\nContext: Tom suggested to Mary to be her editor.\n""",
           "Question: Who will be an editor, Tom or Mary?"]
prompt_text = "Answer: "

prompt = GPTPrompt(context, prompt_text)

replicants = 20
class_lookups = {"Tom": ["tom"], "Mary": ["mary"]}
metric = StringMetric(class_lookups)
text = str(prompt)
kwargs = {"max_tokens": 2, "temperature": 0.5}
metric, responses = run_experiment(run_gpt_prompt, text, replicants, metric, kwargs)
counts, classes = metric.get_metric()

print(counts)
print(responses)



100%|██████████| 20/20 [00:07<00:00,  2.63it/s]

{'Tom': 1, 'Mary': 4, 'other': 15}
[('You will be given a context and a question. Answer the question with either "Tom" or "Mary".\nContext: Tom suggested to Mary to be her editor.\n\nQuestion: Who will be an editor, Tom or Mary?\nAnswer: ', 'Mary'), ('You will be given a context and a question. Answer the question with either "Tom" or "Mary".\nContext: Tom suggested to Mary to be her editor.\n\nQuestion: Who will be an editor, Tom or Mary?\nAnswer: ', '\n'), ('You will be given a context and a question. Answer the question with either "Tom" or "Mary".\nContext: Tom suggested to Mary to be her editor.\n\nQuestion: Who will be an editor, Tom or Mary?\nAnswer: ', '\n'), ('You will be given a context and a question. Answer the question with either "Tom" or "Mary".\nContext: Tom suggested to Mary to be her editor.\n\nQuestion: Who will be an editor, Tom or Mary?\nAnswer: ', '"'), ('You will be given a context and a question. Answer the question with either "Tom" or "Mary".\nContext: Tom su

In [33]:

context = ["""You will be given a context and a question. Answer the question with either "Tom" or "Mary".\nContext: Tom suggested Mary to be his editor.\n""",
           "Question:  Who will be an editor, Tom or Mary?"]
prompt_text = "Answer: "

prompt = GPTPrompt(context, prompt_text)

replicants = 20
class_lookups = {"Tom": ["tom"], "Mary": ["mary"]}
metric = StringMetric(class_lookups)
text = str(prompt)
kwargs = {"max_tokens": 2, "temperature": 0.0}
metric, responses = run_experiment(run_gpt_prompt, text, replicants, metric, kwargs)
counts, classes = metric.get_metric()

print(counts)

100%|██████████| 20/20 [00:09<00:00,  2.22it/s]

{'Tom': 6, 'Mary': 14, 'other': 0}


## Does GPT3 hate object control? 

Let's test a bunch of different object control verbs:







In [54]:
object_control_verbs = ["told", "ordered", "exhorted", "begged", "entreated", "called upon", "reminded", "urged", "asked", "pleaded with", "persuaded", "convinced", "forced", "pushed"]

actions = [("to leave", "left"), ("to call home", "called home"), ("to reply", "replied")]

responses_by_verb = {}

metric = StringMetric(class_lookups)
class_lookups = {"Tom": ["tom"], "Mary": ["mary"]}
for verb in object_control_verbs:
    for infinitive, past in actions:
        context = [f"""You will be given a context and a question. Answer the question with either "Tom" or "Mary".\nContext: Tom {verb} Mary {infinitive}.\n""",
                f"Question:  Who {past}, Tom or Mary?"]
        prompt_text = "Answer: "

        key = f"{verb}, {infinitive}"

        prompt = GPTPrompt(context, prompt_text)

        replicants = 20
        text = str(prompt)
        kwargs = {"max_tokens": 2, "temperature": 0.0}
        metric, responses = run_experiment(run_gpt_prompt, text, replicants, metric, kwargs)
        responses_by_verb[key] = responses 

counts, classes = metric.get_metric()
print(counts)



100%|██████████| 20/20 [00:07<00:00,  2.59it/s]

{'Tom': 214, 'Mary': 602, 'other': 24}


In [62]:
for key, responses in responses_by_verb.items():
    mary = 0
    tom = 0
    other = 0
    for __, r in responses:
        if r.strip().lower() == "mary": 
            mary += 1
        elif r.strip().lower() == "tom": 
            tom += 1
        else:
            other += 1
    if tom > 0:
        print(f"{key}: Tom: {tom}, Mary: {mary}")


told, to call home: Tom: 11, Mary: 9
ordered, to call home: Tom: 9, Mary: 11
exhorted, to call home: Tom: 2, Mary: 18
begged, to leave: Tom: 6, Mary: 14
begged, to call home: Tom: 18, Mary: 2
entreated, to call home: Tom: 8, Mary: 12
called upon, to leave: Tom: 3, Mary: 17
called upon, to call home: Tom: 20, Mary: 0
reminded, to call home: Tom: 15, Mary: 5
urged, to call home: Tom: 6, Mary: 14
asked, to call home: Tom: 17, Mary: 3
asked, to reply: Tom: 2, Mary: 14
pleaded with, to call home: Tom: 5, Mary: 15
persuaded, to leave: Tom: 10, Mary: 10
persuaded, to call home: Tom: 20, Mary: 0
persuaded, to reply: Tom: 1, Mary: 19
convinced, to leave: Tom: 5, Mary: 15
convinced, to call home: Tom: 18, Mary: 2
forced, to leave: Tom: 10, Mary: 10
forced, to call home: Tom: 16, Mary: 4
pushed, to call home: Tom: 12, Mary: 8
